In [1]:
print 'load libraries...'

load libraries...


In [2]:
# Aim: 

In [3]:
# Add path in order to load utility functions
import sys
sys.path.append(r"N:\18. Hot Spot\Custom Modules\Utils")
sys.path.append(r"N:\18. Hot Spot\Custom Modules\ETLMonthly")

In [11]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from collections import OrderedDict
path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
import file_utils
import db_utils
from sqlalchemy import create_engine
import config
#set working directory
os.chdir(config.working_dir)

In [12]:
pd.set_option('display.max_columns', 500)

In [13]:
# db credentials authentication

In [14]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username,config.db_pwd)

connect to database...
Testing Connection...
Connection Succesful!


In [17]:
mtch = re.match('.*@', conn_string)
un, pwd = mtch.group().rstrip('@').split('/')

In [18]:
con = cx_Oracle.connect(conn_string)
cs_retail_loan_cust_data = pd.read_sql_query('select * from cs_retail_loan_cust', con)
con.close()

#### Get GTWorld trxns for retail loan customers only and insert into database

In [24]:
for i, f in enumerate(file_utils.get_files(r"N:\10. Executive\Original\ETL\gtworld_trxns")):
    print i+1, '. ', f
    cur_df = file_utils.unzip(f)
    cur_df = cs_retail_loan_cust_data[['BRA_CODE','CUS_NUM']].merge(
        cur_df, on=['BRA_CODE','CUS_NUM'], how='inner').fillna(0)
        
    if_exists_var = 'append'
    if i==0:
        if_exists_var = 'replace'
    
    cur_df['REFERENCE_DATE'] = pd.to_datetime(cur_df['REFERENCE_DATE'], format="%d%b%Y")
    cur_df['CTRL_DATE'] = pd.to_datetime(cur_df['CTRL_DATE'], format="%d%b%Y")
    
    conn = create_engine('oracle+cx_oracle://{}:{}@dwdrscan.gtbank.com:1521/?service_name=odidb'.format(un, pwd))    
    cur_df.to_sql('cs_retail_loans_gtworld', conn, index=False, if_exists=if_exists_var, chunksize=10000,
                                                                      dtype=db_utils.sqlcol(cur_df))
    del cur_df

1 .  N:\10. Executive\Original\ETL\gtworld_trxns\04Mar2020.csv.gz
2 .  N:\10. Executive\Original\ETL\gtworld_trxns\28Feb2018.csv.gz
3 .  N:\10. Executive\Original\ETL\gtworld_trxns\28Feb2019.csv.gz
4 .  N:\10. Executive\Original\ETL\gtworld_trxns\29Feb2020.csv.gz
5 .  N:\10. Executive\Original\ETL\gtworld_trxns\30Apr2018.csv.gz
6 .  N:\10. Executive\Original\ETL\gtworld_trxns\30Apr2019.csv.gz
7 .  N:\10. Executive\Original\ETL\gtworld_trxns\30Jun2018.csv.gz
8 .  N:\10. Executive\Original\ETL\gtworld_trxns\30Jun2019.csv.gz
9 .  N:\10. Executive\Original\ETL\gtworld_trxns\30Nov2018.csv.gz
10 .  N:\10. Executive\Original\ETL\gtworld_trxns\30Nov2019.csv.gz
11 .  N:\10. Executive\Original\ETL\gtworld_trxns\30Sep2018.csv.gz
12 .  N:\10. Executive\Original\ETL\gtworld_trxns\30Sep2019.csv.gz
13 .  N:\10. Executive\Original\ETL\gtworld_trxns\31Aug2018.csv.gz
14 .  N:\10. Executive\Original\ETL\gtworld_trxns\31Aug2019.csv.gz
15 .  N:\10. Executive\Original\ETL\gtworld_trxns\31Dec2018.csv.gz
16 .

#### Get Retail Customer Digital Trxns

In [25]:

db_utils.create_table_from_selectquery(
'cs_digital_trxns',
'''
select reference_date, b.bra_code, b.cus_num, cnt, vol, cast(channel as NVARCHAR2(50)) channel
from data_analyst.cs_retail_loan_cust a
inner join
(select * from stg.d_chnl_trx@exadata_new where channel!='GTWORLD') b
on a.bra_code=b.bra_code and a.cus_num=b.cus_num
and b.reference_date between trunc(add_months(a.date_open, -12),'month') and trunc(add_months(a.date_open, -1),'month')

union all

select  reference_date, b.bra_code, b.cus_num, cnt, vol, cast(channel as NVARCHAR2(50)) channel
from data_analyst.cs_retail_loan_cust a
inner join
(select * from data_analyst.cs_retail_loans_gtworld) b
on a.bra_code=b.bra_code and a.cus_num=b.cus_num
and b.reference_date between trunc(add_months(a.date_open, -12),'month') and trunc(add_months(a.date_open, -1),'month')
''',
conn_string, drop_if_exists=True)

Creating New Table...
New table records:  4598465


In [21]:
# get value and count of the eChannel trxns

In [26]:
con = cx_Oracle.connect(conn_string)
df_digital_trxns = pd.read_sql_query('''
select bra_code, cus_num,
sum(case when channel='737' then cnt else 0 end) USSD_cnt,
sum(case when channel='737' then vol else 0 end) USSD_vol,
sum(case when channel='gaps' then cnt else 0 end) GAPS_cnt,
sum(case when channel='gaps' then vol else 0 end) GAPS_vol,
sum(case when channel='pos_wdr' then cnt else 0 end) POS_WDR_cnt,
sum(case when channel='pos_wdr' then vol else 0 end) POS_WDR_vol,
sum(case when channel='atm_rou' then cnt else 0 end) ATM_ROU_cnt,
sum(case when channel='atm_rou' then vol else 0 end) ATM_ROU_vol,
sum(case when channel='atm_on_us' then cnt else 0 end) ATM_ON_US_cnt,
sum(case when channel='atm_on_us' then vol else 0 end) ATM_ON_US_vol,
sum(case when channel='pos' then cnt else 0 end) POS_cnt,
sum(case when channel='pos' then vol else 0 end) POS_vol,
sum(case when channel='MB' then cnt else 0 end) MB_cnt,
sum(case when channel='MB' then vol else 0 end) MB_vol,
sum(case when channel='IB' then cnt else 0 end) IB_cnt,
sum(case when channel='IB' then vol else 0 end) IB_vol,
sum(case when channel='GTWORLD' then cnt else 0 end) GTWORLD_cnt,
sum(case when channel='GTWORLD' then vol else 0 end) GTWORLD_vol
from cs_digital_trxns
group by bra_code, cus_num
''', con)
con.close()

print len(df_digital_trxns)

153490


In [27]:
con = cx_Oracle.connect(conn_string)
cs_retail_loan_cust = pd.read_sql_query('select bra_code, cus_num from cs_retail_loan_cust', con)
con.close()

len(cs_retail_loan_cust)

154724

In [28]:
cs_retail_loan_cust.merge(df_digital_trxns, on=['BRA_CODE','CUS_NUM'], 
                          how='left').fillna(0).to_csv('cs_digital_trxns_info.csv', index=False)